# GEPA for JEE (Math)
Using the `dspy.GEPA` optimizer, we compare the performances of Gemma 27b It and OpenAI OSS 20b on Math questions from JEE, before and after GEPA optmization.

In [ ]:
# api_key = input("Enter your OpenAI API key: ")
import dspy
import re

api_key = "your_api_key"
gemini_api_key = "your_api_key"

gemma = "openrouter/google/gemma-3-27b-it"
qwen = "openrouter/qwen/qwen3-32b"
openai_oss = "openai/gpt-oss-20b"

      
extraction_lm = dspy.LM(
    model="openrouter/openai/gpt-4o-mini", 
    api_base="https://openrouter.ai/api/v1",
    api_key=api_key, 
    temperature=0.1, 
    max_tokens=32000
)

from dspy.adapters import TwoStepAdapter
adapter = dspy.adapters.TwoStepAdapter(extraction_lm)

lm = dspy.LM(
    model=gemma,
    api_base="https://openrouter.ai/api/v1",
    api_key=api_key,
    temperature=1,
    max_tokens=32000,
)

dspy.configure(lm=lm, adapter=adapter)

In [53]:
import dspy

# Define a signature for feedback
class FeedbackSignature(dspy.Signature):
    """Give feedback on an incorrect solution to a math problem. It might be incorrect due to the reasoning or the answer formatting,
    solution must be inside \boxed{}. solution must follow this format: problem = dspy.InputField()
    answer = dspy.OutputField()
    """
    question: str = dspy.InputField()
    solution: str = dspy.InputField()
    expected_answer: str = dspy.InputField()
    feedback: str = dspy.OutputField(desc="Constructive feedback about the solution")

gemini_lm = dspy.LM(
    model="gemini-2.5-flash-lite",
    temperature=1.0,
    max_tokens=32000,
    api_key=gemini_api_key
)

# Function to get feedback
def get_llm_feedback(question, solution, expected_answer):
    # Create the feedback module
    feedback_module = dspy.ChainOfThought(
        FeedbackSignature,
        lm=gemini_lm
    )
    
    with dspy.context(lm=gemini_lm):
        # Call the LLM with structured inputs
        result = feedback_module(
            question=question,
            solution=solution,
            expected_answer=expected_answer
    )
        
    print("Feedback:", result.feedback[:100])
    
    return result.feedback


In [54]:
!pip install google-auth


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [55]:
get_llm_feedback("How much is 2 + 2?", "The answer is 4.", "4")  # Example usage

Feedback: The solution is correct and follows the expected format. No changes are needed.


'The solution is correct and follows the expected format. No changes are needed.'

In [56]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals 2?")

Prediction(
    reasoning='When two dice are tossed, the sample space consists of 36 possible outcomes, since each die has 6 faces. We want to find the probability that the sum of the two dice equals 2. The only way to get a sum of 2 is if both dice show a 1. This corresponds to the outcome (1, 1). Therefore, there is only 1 favorable outcome. The probability of an event is calculated as the number of favorable outcomes divided by the total number of possible outcomes. In this case, the probability is 1/36.',
    answer=0.027777777777777776
)

### Loading the JEE dataset

In [ ]:
import dspy
from datasets import load_dataset

def init_jee_dataset():
    dataset = load_dataset("PhysicsWallahAI/JEE-Main-2025-Math", "apr")['test']

    # Split the dataset
    split = dataset.train_test_split(test_size=0.25, seed=42)
    train_set_2025 = split['train']
    test_set_2025 = split['test'].select(range(int(0.5 * len(split['test']))))
    val_set_2025 = split['test'].select(range(int(0.5 * len(split['test'])), len(split['test'])))

    # Convert to dspy.Example format
    train_set = [
        dspy.Example({
            "problem": x['question'],
            "answer": x['answer'],
        }).with_inputs("problem")
        for x in train_set_2025
    ]

    val_set = [
        dspy.Example({
            "problem": x['question'],
            "answer": x['answer'],
        }).with_inputs("problem")
        for x in val_set_2025
    ]

    test_set = [
        dspy.Example({
            "problem": x['question'],
            "answer": x['answer'],
        }).with_inputs("problem")
        for x in test_set_2025
    ]

    return train_set, val_set, test_set

# Example usage
train, val, test = init_jee_dataset()
print("Train size:", len(train))
print("Val size:", len(val))
print("Test size:", len(test))


Train size: 168
Val size: 29
Test size: 28


### Defining the program: `dspy.ChainOfThought`

In [ ]:
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)
program_openai = dspy.ChainOfThought(GenerateResponse)

### Defining the evaluation metric
We simply check exact match between the predicted answer and the correct answer.

In [61]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = str(example['answer'])
    try:
        llm_answer = str(prediction.answer)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

### Optimize the program with `dspy.GEPA`

In [ ]:
#Experimental: adding a LLm-as-judge to provide feedback
def metric_with_feedback_llm(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = str(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = str(prediction.answer)
    except ValueError as e:
        print(f"Couldn't parse answer as integer: {prediction.answer}")
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        #feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
        #    feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
            feedback_text += get_llm_feedback(example['problem'], written_solution, str(correct_answer))
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    feedback_text = get_llm_feedback(example['problem'], written_solution, str(correct_answer))
    # feedback_text = ""
    # if score == 1:
    #     feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    # else:
    #     feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    # if written_solution:
    #     feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

In [63]:
def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    
    correct_answer = str(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = str(prediction.answer)
    except ValueError as e:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)
    
    if llm_answer is None:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    # if not score and pred_name is not None:
    #     print("Calling the llm feedback function!")
    #     return metric_with_feedback_llm(example, prediction, trace, pred_name, pred_trace)
    # else:
    feedback_text = ""
    if score == 1:
        feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    else:
        feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    if written_solution:
        feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

In [45]:
# Extract candidate data with prompts and scores for analysis
if hasattr(optimized_program_openai, 'detailed_results'):
    results = optimized_program_openai.detailed_results
    
    # Create structured candidate data
    candidate_data = []
    
    for i, (candidate, score) in enumerate(zip(results.candidates, results.val_aggregate_scores)):
        for name, predictor in candidate.named_predictors():
            prompt = predictor.signature.instructions
            
            candidate_entry = {
                'candidate_idx': i,
                'score': score,
                'prompt': prompt,
                'prompt_length_chars': len(prompt),
                'prompt_length_words': len(prompt.split()),
                'predictor_name': name
            }
            
            candidate_data.append(candidate_entry)
            break  # Just take first predictor for each candidate
    
    # Print summary
    print("📋 GEPA CANDIDATE DATA EXTRACTED")
    print("=" * 50)
    print(f"Total candidates: {len(candidate_data)}")
    
    # Show best and worst performers
    best_candidate = max(candidate_data, key=lambda x: x['score'])
    worst_candidate = min(candidate_data, key=lambda x: x['score'])
    
    print(f"\n🏆 Best candidate:")
    print(f"  Index: {best_candidate['candidate_idx']}")
    print(f"  Score: {best_candidate['score']:.4f}")
    print(f"  Length: {best_candidate['prompt_length_chars']} chars, {best_candidate['prompt_length_words']} words")
    
    print(f"\n📉 Worst candidate:")
    print(f"  Index: {worst_candidate['candidate_idx']}")
    print(f"  Score: {worst_candidate['score']:.4f}")
    print(f"  Length: {worst_candidate['prompt_length_chars']} chars, {worst_candidate['prompt_length_words']} words")
    
    # Show score progression
    scores = [c['score'] for c in candidate_data]
    print(f"\n📈 Score progression: {scores[0]:.3f} → {scores[-1]:.3f} (change: {scores[-1] - scores[0]:+.3f})")
    
    print(f"\n💡 Usage examples:")
    print(f"  Best prompt: candidate_data[{best_candidate['candidate_idx']}]['prompt']")
    print(f"  All scores: [c['score'] for c in candidate_data]")
    print(f"  High performers: [c for c in candidate_data if c['score'] > 0.5]")
    
else:
    print("No detailed results found. Make sure GEPA was run with track_stats=True")

📋 GEPA CANDIDATE DATA EXTRACTED
Total candidates: 10

🏆 Best candidate:
  Index: 4
  Score: 0.6207
  Length: 1226 chars, 188 words

📉 Worst candidate:
  Index: 1
  Score: 0.1379
  Length: 2343 chars, 342 words

📈 Score progression: 0.276 → 0.517 (change: +0.241)

💡 Usage examples:
  Best prompt: candidate_data[4]['prompt']
  All scores: [c['score'] for c in candidate_data]
  High performers: [c for c in candidate_data if c['score'] > 0.5]


In [46]:
import json

with open("data_openai.json", "w") as json_file:
    json.dump(candidate_data, json_file, indent=4)  # indent makes it more readable

In [30]:
candidate_data

[{'candidate_idx': 0,
  'score': 0.4482758620689655,
  'prompt': 'Solve the problem and provide the answer in the correct format.',
  'prompt_length_chars': 63,
  'prompt_length_words': 11,
  'predictor_name': 'predict'},
 {'candidate_idx': 1,
  'score': 0.41379310344827586,
  'prompt': "You are a math problem-solving assistant. Your goal is to accurately solve the provided mathematical problem and present the solution in a specific two-part format.\n\n**Task Instructions:**\n\n1.  **Solve the Problem:** Carefully read the problem, identify the key mathematical concepts involved (e.g., calculus, algebra, probability, number theory), and determine the correct method to find the solution.\n2.  **Format the Output:** Your response must contain exactly two sections, in this order:\n    *   `### reasoning`\n    *   `### answer`\n\n**Section Details:**\n\n1.  **`### reasoning`**:\n    *   In this section, provide a clear explanation of your problem-solving process.\n    *   You can either gi

In [47]:
# Examine candidate_data structure for ancestry tracking
print(f"Total candidates: {len(candidate_data)}")
print(f"First candidate keys: {list(candidate_data[0].keys())}")
print(f"Sample candidate structure:")
import json
print(json.dumps(candidate_data[0], indent=2)[:1000] + "..." if len(str(candidate_data[0])) > 1000 else json.dumps(candidate_data[0], indent=2))

Total candidates: 10
First candidate keys: ['candidate_idx', 'score', 'prompt', 'prompt_length_chars', 'prompt_length_words', 'predictor_name']
Sample candidate structure:
{
  "candidate_idx": 0,
  "score": 0.27586206896551724,
  "prompt": "Solve the problem and provide the answer in the correct format.",
  "prompt_length_chars": 63,
  "prompt_length_words": 11,
  "predictor_name": "predict"
}


In [70]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    #max_full_evals=2,
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=5,
    reflection_lm=dspy.LM(model="gemini-2.5-pro", temperature=1.0, max_tokens=32000, api_key=gemini_api_key)
)

optimized_program = optimizer.compile(
    program,
    trainset=train,
    valset=val,
)

2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 496 metric calls of the program. This amounts to 2.52 full evals on the train+val set.
2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Using 29 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Using 29 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
2025/08/20 21:22:44 INFO dspy.evaluate.evaluate: Average Metric: 13.0 / 29 (44.8%)
2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4482758620689655
2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4482758620689655
2025/08/20 21:22:44 INFO dspy.evaluate.evaluate: Average Metric: 13.0 / 29 (44.8

Average Metric: 4.00 / 5 (80.0%): 100%|██████████| 5/5 [00:00<00:00, 4633.57it/s] 

2025/08/20 21:22:44 INFO dspy.evaluate.evaluate: Average Metric: 4.0 / 5 (80.0%)
2025/08/20 21:22:44 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You are a math problem-solving assistant. Your goal is to accurately solve the provided mathematical problem and present the solution in a specific two-part format.

**Task Instructions:**

1.  **Solve the Problem:** Carefully read the problem, identify the key mathematical concepts involved (e.g., calculus, algebra, probability, number theory), and determine the correct method to find the solution.
2.  **Format the Output:** Your response must contain exactly two sections, in this order:
    *   `### reasoning`
    *   `### answer`

**Section Details:**

1.  **`### reasoning`**:
    *   In this section, provide a clear explanation of your problem-solving process.
    *   You can either give a high-level summary of your approach or a detailed, step-by-step derivation.
    *   Mention the key mathematical princip


Average Metric: 1.00 / 5 (20.0%): 100%|██████████| 5/5 [00:00<00:00, 3903.13it/s]

2025/08/20 21:22:45 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 5 (20.0%)
2025/08/20 21:22:45 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Proposed new text for predict: You are an expert math problem-solving assistant. Your primary goal is to solve the given mathematical problem with high accuracy and present the solution in a specific, clean, two-part format.

**Task Instructions:**

1.  **Analyze and Solve:**
    *   Read the problem meticulously. Identify the mathematical domain (e.g., calculus, trigonometry, algebra, probability, geometry).
    *   Select the most effective strategy. Think step-by-step and be extremely careful with algebraic manipulations, substitutions, and applying formulas.
    *   **Double-check your work for common pitfalls.** For example:
        *   **Trigonometry:** Remember that `sqrt(x^2) = |x|`. Be mindful of the signs of trigonometric functions in different quadrants (e.g., `cos(2)` is negative since 2 radians is in the second quadrant). Pay clo

...

2025/08/20 21:22:46 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 5 (0.0%)
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 28: New subsample score is not better, skipping
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 28: New subsample score is not better, skipping
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Selected program 6 score: 0.4482758620689655
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Selected program 6 score: 0.4482758620689655


Average Metric: 2.00 / 5 (40.0%): 100%|██████████| 5/5 [00:00<00:00, 1053.74it/s]

2025/08/20 21:22:46 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 5 (40.0%)
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Proposed new text for predict: You are an expert math problem-solving assistant. Your primary goal is to provide accurate, rigorously-derived solutions to mathematical problems, presented in a specific two-part format. Your derivations must be methodical, transparent, and verifiable.

**Task Instructions:**

1.  **Analyze and Solve:**
    *   **Deconstruct the Problem:** Carefully read the problem to identify all given information, constraints, and the specific question being asked.
    *   **Identify Key Concepts:** Determine the relevant mathematical field(s) (e.g., calculus, algebra, number theory) and the specific techniques required.
    *   **Formulate a Strategy:** Select an appropriate and systematic method for solving the problem.
    *   **Execute and Rigorously Verify:** Carry out the steps of your chosen method. This is the most 


Average Metric: 1.00 / 5 (20.0%): 100%|██████████| 5/5 [00:00<00:00, 1042.32it/s]

2025/08/20 21:22:46 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 5 (20.0%)
2025/08/20 21:22:46 INFO dspy.teleprompt.gepa.gepa: Iteration 30: Proposed new text for predict: You are an expert mathematician and a clear communicator. Your task is to solve a given mathematical problem and present the solution in a structured, two-part format. Your primary goal is to be accurate and to provide a clear, easy-to-follow explanation of your method.

**Task Instructions:**

1.  **Solve the Problem:** Meticulously analyze the problem to identify the underlying mathematical concepts. Devise a correct and efficient strategy for the solution.
    *   Look for elegant or efficient methods (e.g., using geometric properties, specialized theorems) over brute-force calculations where possible.
    *   Double-check all formulas, properties, and calculations before finalizing your reasoning.
    *   **Crucially, at the end of your derivation, re-read the problem statement one last time to ensure your 

It can be seen that what GEPA is doing here, is precomputing some reasoning to come up with a good plan for future task instances. Due to the improved performance in unseen validation set, we expect this prompt to generalize!

In [ ]:
import dspy
evaluate = dspy.Evaluate(
    devset=test,
    metric=metric,
    num_threads=1,
    display_table=True,
    display_progress=True,
    provide_traceback=True
)

evaluate(program)

Average Metric: 7.00 / 24 (29.2%):  86%|████████▌ | 24/28 [07:15<01:30, 22.68s/it]

2025/08/19 23:53:23 ERROR dspy.utils.parallelizer: Error for Example({'problem': 'If \n\\[ \\int \\left( \\frac{1}{x} + \\frac{1}{x^3} \\right) \\left( \\sqrt{3x^{-24} + x^{-26}} \\right) dx \\] \nis equal to \n\\[ \\frac{\\alpha}{3(\\alpha + 1)} \\left(3x^{\\beta} + x^{\\gamma}\\right)^{\\frac{\\alpha + 1}{\\alpha}} + C,\\ x > 0, \\] \nwhere \\(\\alpha, \\beta, \\gamma \\in \\mathbb{Z}\\), and \\(C\\) is the constant of integration, then \\(\\alpha + \\beta + \\gamma\\) is equal to:', 'answer': '19'}) (input_keys={'problem'}): litellm.InternalServerError: InternalServerError: OpenAIException - error - Expecting value: line 161 column 1 (char 880), Received response - <APIResponse [200 OK] type=<class 'openai.types.chat.chat_completion.ChatCompletion'>>, Type of response - <class 'openai._legacy_response.LegacyAPIResponse'>
Traceback (most recent call last):
  File "/Users/lucasfernandesmartins/Documents/gemma_garage/venv/lib/python3.13/site-packages/litellm/llms/openai/openai.py", lin

Average Metric: 8.00 / 27 (29.6%): 100%|██████████| 28/28 [12:56<00:00, 27.75s/it]

2025/08/19 23:53:23 INFO dspy.evaluate.evaluate: Average Metric: 8.0 / 28 (28.6%)


,problem,example_answer,reasoning,pred_answer,metric,answer
0,"Let \(P_n=\alpha^n+\beta^n,\ n\in\mathbb{N}\). If \(P_{10}=123,\ P...",x^2 + x - 1 = 0,None,None,,NaN
1,If the range of the function \(f(x) = \frac{5 - x}{x^2 - 3x + 2}\)...,194,"For \( f(x)=\frac{5-x}{x^2-3x+2}, \qquad x\neq1,2, \) set \(y=f(x)...",\[\n\boxed{194}\n\],,NaN
2,"Let the function \( f(x) = \frac{x}{3} + \frac{3}{x} + 3, \; x \ne...",36,None,None,,NaN
3,"Let $A = \{1, 2, 3, \ldots, 10\}$ and $R$ be a relation on $A$ suc...",5,"1. **Determine the relation \( R \)** \( R = \{(a,b)\in A\times A ...",\[\n\boxed{2}\n\],,NaN
4,Let the domains of the functions $f(x)=\log_{10}\log_{10}\bigl(8-\...,15,The problem involves finding the domains of two functions \(f(x)\)...,267,,NaN
5,Let the product of \(\omega_1 = (8+i)\sin\theta + (7+4i)\cos\theta...,130,"Let \[ \omega_1=(8+i)\sin\theta+(7+4i)\cos\theta, \qquad \omega_2=...",130,✔️ [1.000],NaN
6,Let \(f:\mathbb{R}\to\mathbb{R}\) be a thrice‐differentiable odd f...,36,The differential equation is \(f''(x)=f(x)\). Its general solution...,\[\n\boxed{36}\n\],,NaN
7,"For \(\alpha,\beta,\gamma\in\mathbb{R}\), if \(\displaystyle\lim_{...",7,We need to evaluate the limit \[ \lim_{x\to0}\frac{x^2\sin(\alpha ...,7,✔️ [1.000],NaN
8,"Let $A=\{-2,-1,0,1,2,3\}$. Define a relation $R$ on $A$ by $xRy$ i...",12,"The relation is defined on \(A=\{-2,-1,0,1,2,3\}\) by \(x\,R\,y \i...",12,✔️ [1.000],NaN
9,If \lim_{x\to1} \frac{(x-1)(6 + \lambda \cos(x-1)) + \mu \sin(1-x)...,18,None,None,,NaN


EvaluationResult(score=28.57, results=<list of 28 results>)

In [ ]:
import dspy
evaluate = dspy.Evaluate(
    devset=test,
    metric=metric,
    num_threads=1,
    display_table=True,
    display_progress=True,
    provide_traceback=True
)

evaluate(optimized_program)

Average Metric: 15.00 / 24 (62.5%):  86%|████████▌ | 24/28 [08:30<02:00, 30.25s/it]

2025/08/20 21:17:49 ERROR dspy.utils.parallelizer: Error for Example({'problem': 'If \n\\[ \\int \\left( \\frac{1}{x} + \\frac{1}{x^3} \\right) \\left( \\sqrt{3x^{-24} + x^{-26}} \\right) dx \\] \nis equal to \n\\[ \\frac{\\alpha}{3(\\alpha + 1)} \\left(3x^{\\beta} + x^{\\gamma}\\right)^{\\frac{\\alpha + 1}{\\alpha}} + C,\\ x > 0, \\] \nwhere \\(\\alpha, \\beta, \\gamma \\in \\mathbb{Z}\\), and \\(C\\) is the constant of integration, then \\(\\alpha + \\beta + \\gamma\\) is equal to:', 'answer': '19'}) (input_keys={'problem'}): litellm.InternalServerError: InternalServerError: OpenAIException - error - Expecting value: line 435 column 1 (char 2387), Received response - <APIResponse [200 OK] type=<class 'openai.types.chat.chat_completion.ChatCompletion'>>, Type of response - <class 'openai._legacy_response.LegacyAPIResponse'>
Traceback (most recent call last):
  File "/Users/lucasfernandesmartins/Documents/gemma_garage/venv/lib/python3.13/site-packages/litellm/llms/openai/openai.py", li

Average Metric: 16.00 / 27 (59.3%): : 29it [17:01, 35.24s/it]                       

2025/08/20 21:20:54 INFO dspy.evaluate.evaluate: Average Metric: 16.0 / 28 (57.1%)


,problem,example_answer,reasoning,pred_answer,metric,answer
0,"Let \(P_n=\alpha^n+\beta^n,\ n\in\mathbb{N}\). If \(P_{10}=123,\ P...",x^2 + x - 1 = 0,The equation x^2 + x - 1 = 0 is a quadratic equation. To find the ...,The solutions to the equation x^2 + x - 1 = 0 are x = (-1 + √5) / ...,,NaN
1,If the range of the function \(f(x) = \frac{5 - x}{x^2 - 3x + 2}\)...,194,The reasoning provided outlines the steps to determine the range o...,194,✔️ [1.000],NaN
2,"Let the function \( f(x) = \frac{x}{3} + \frac{3}{x} + 3, \; x \ne...",36,18,18,,NaN
3,"Let $A = \{1, 2, 3, \ldots, 10\}$ and $R$ be a relation on $A$ suc...",5,"The input provided is simply the number ""2"", which does not contai...","There is no specific answer to provide based on the input ""2"".",,NaN
4,Let the domains of the functions $f(x)=\log_{10}\log_{10}\bigl(8-\...,15,267,267,,NaN
5,Let the product of \(\omega_1 = (8+i)\sin\theta + (7+4i)\cos\theta...,130,130,130,✔️ [1.000],NaN
6,Let \(f:\mathbb{R}\to\mathbb{R}\) be a thrice‐differentiable odd f...,36,36,36,✔️ [1.000],NaN
7,"For \(\alpha,\beta,\gamma\in\mathbb{R}\), if \(\displaystyle\lim_{...",7,We expand the numerator and denominator of the given limit as \(x\...,7,✔️ [1.000],NaN
8,"Let $A=\{-2,-1,0,1,2,3\}$. Define a relation $R$ on $A$ by $xRy$ i...",12,The reasoning provided outlines the steps taken to analyze the rel...,12,✔️ [1.000],NaN
9,If \lim_{x\to1} \frac{(x-1)(6 + \lambda \cos(x-1)) + \mu \sin(1-x)...,18,"Let \[ h=x-1,\qquad h\to 0 . \] Then \[ \cos (x-1)=\cos h,\qquad \...",18,✔️ [1.000],NaN


EvaluationResult(score=57.14, results=<list of 28 results>)

In [ ]:
# After running GEPA with track_stats=True
if hasattr(optimized_program, 'detailed_results'):
    results = optimized_program.detailed_results
    
    # Extract data for plotting
    prompt_chars = []
    prompt_words = []
    scores = []
    candidate_nums = []
    
    for i, (candidate, score) in enumerate(zip(results.candidates, results.val_aggregate_scores)):
        for name, predictor in candidate.named_predictors():
            prompt = predictor.signature.instructions
            prompt_chars.append(len(prompt))
            prompt_words.append(len(prompt.split()))
            scores.append(score)
            candidate_nums.append(i)
            break  # Just take first predictor
    
    # Create the plots
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Score vs Character Count
    scatter1 = ax1.scatter(prompt_chars, scores, c=candidate_nums, cmap='viridis', alpha=0.7, s=60)
    ax1.set_xlabel('Prompt Length (characters)')
    ax1.set_ylabel('Validation Score')
    ax1.set_title('GEPA: Score vs Prompt Character Count')
    ax1.grid(True, alpha=0.3)
    
    # Add correlation coefficient
    if len(prompt_chars) > 1:
        correlation_chars = np.corrcoef(prompt_chars, scores)[0, 1]
        ax1.text(0.05, 0.95, f'Correlation: {correlation_chars:.3f}', 
                transform=ax1.transAxes, 
                bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.8))
    
    # Add best candidate annotation
    best_idx = results.best_idx
    best_char_count = prompt_chars[best_idx]
    best_score = scores[best_idx]
    ax1.scatter([best_char_count], [best_score], c='red', s=100, marker='*', 
               label=f'Best (#{best_idx})', edgecolor='black', linewidth=1)
    ax1.legend()
    
    # Plot 2: Score vs Word Count
    scatter2 = ax2.scatter(prompt_words, scores, c=candidate_nums, cmap='viridis', alpha=0.7, s=60)
    ax2.set_xlabel('Prompt Length (words)')
    ax2.set_ylabel('Validation Score')
    ax2.set_title('GEPA: Score vs Prompt Word Count')
    ax2.grid(True, alpha=0.3)
    
    # Add correlation coefficient
    if len(prompt_words) > 1:
        correlation_words = np.corrcoef(prompt_words, scores)[0, 1]
        ax2.text(0.05, 0.95, f'Correlation: {correlation_words:.3f}', 
                transform=ax2.transAxes, 
                bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.8))
    
    # Add best candidate annotation
    best_word_count = prompt_words[best_idx]
    ax2.scatter([best_word_count], [best_score], c='red', s=100, marker='*', 
               label=f'Best (#{best_idx})', edgecolor='black', linewidth=1)
    ax2.legend()
    
    # Plot 3: Evolution Timeline - Character Count
    ax3.plot(candidate_nums, prompt_chars, 'b-o', markersize=4, alpha=0.7)
    ax3.scatter([best_idx], [best_char_count], c='red', s=100, marker='*', 
               label=f'Best (#{best_idx})', edgecolor='black', linewidth=1)
    ax3.set_xlabel('Candidate Number (Evolution Order)')
    ax3.set_ylabel('Prompt Length (characters)')
    ax3.set_title('Prompt Length Evolution Over Time')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # Plot 4: Evolution Timeline - Scores
    ax4.plot(candidate_nums, scores, 'g-o', markersize=4, alpha=0.7)
    ax4.scatter([best_idx], [best_score], c='red', s=100, marker='*', 
               label=f'Best (#{best_idx})', edgecolor='black', linewidth=1)
    ax4.set_xlabel('Candidate Number (Evolution Order)')
    ax4.set_ylabel('Validation Score')
    ax4.set_title('Score Evolution Over Time')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # Add colorbar for candidate numbers
    plt.colorbar(scatter1, ax=ax1, label='Candidate #')
    plt.colorbar(scatter2, ax=ax2, label='Candidate #')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("GEPA PROMPT EVOLUTION ANALYSIS")
    print("="*60)
    
    print(f"\nPrompt Length Statistics:")
    print(f"Character count range: {min(prompt_chars)} - {max(prompt_chars)} chars")
    print(f"Word count range: {min(prompt_words)} - {max(prompt_words)} words")
    print(f"Average character count: {np.mean(prompt_chars):.1f} chars")
    print(f"Average word count: {np.mean(prompt_words):.1f} words")
    
    print(f"\nScore Statistics:")
    print(f"Score range: {min(scores):.4f} - {max(scores):.4f}")
    print(f"Average score: {np.mean(scores):.4f}")
    print(f"Score std dev: {np.std(scores):.4f}")
    
    print(f"\nBest Candidate:")
    print(f"Candidate #{best_idx}: Score={best_score:.4f}")
    print(f"Length: {best_char_count} chars, {best_word_count} words")
    
    if len(prompt_chars) > 1:
        print(f"\nCorrelations:")
        print(f"Length (chars) vs Score: {correlation_chars:.3f}")
        print(f"Length (words) vs Score: {correlation_words:.3f}")
        
        # Growth analysis
        initial_chars = prompt_chars[0]
        final_chars = prompt_chars[-1]
        char_growth = final_chars - initial_chars
        char_growth_pct = (char_growth / initial_chars * 100) if initial_chars > 0 else 0
        
        initial_score = scores[0]
        final_score = scores[-1]
        score_change = final_score - initial_score
        
        print(f"\nEvolution Summary:")
        print(f"Character growth: {initial_chars} → {final_chars} ({char_growth:+d} chars, {char_growth_pct:+.1f}%)")
        print(f"Score change: {initial_score:.4f} → {final_score:.4f} ({score_change:+.4f})")
else:
    print("No detailed results found. Make sure to run GEPA with track_stats=True")